In [1]:
journals = [
    # FOOD
    """I spent the evening experimenting in the kitchen.
    I made crispy fries from scratch and seasoned them with rosemary and sea salt.
    Cooking felt slow and grounding, like meditation in motion.
    The best part was sitting down to taste something I made entirely myself.""",

    """I cooked a big pasta meal tonight, just me and the quiet kitchen.
    The smell of garlic and olive oil filled the room.
    Eating alone felt comfortable rather than sad, which surprised me.
    Maybe this is what self-care actually looks like.""",

    # EXERCISE
    """I forced myself to go for a morning run before work.
    The first few minutes were rough, but once my breathing found a rhythm, everything felt lighter.
    The sunrise over the river almost made me forget how tired I was.
    It reminded me that most progress starts with discomfort.""",

    """Today’s workout was heavy on strength training.
    My arms are sore, but it’s the kind of ache that tells me I did something right.
    I tracked my lifts and saw steady improvement compared to last week.
    Discipline feels like a quiet kind of reward.""",

    # SLEEP
    """I finally let myself sleep in without feeling guilty.
    Woke up late and stretched slowly under the blanket.
    The extra hours made me realize how exhausted I’ve been.
    I need to protect rest as much as I protect productivity.""",

    """I took an afternoon nap that turned into a full-on two-hour sleep.
    I woke up disoriented but strangely calm.
    It’s wild how the body decides what it needs even when the mind refuses to stop.""",

    # FRIENDS
    """Met up with Lucas for coffee after weeks of not seeing anyone.
    We talked about small things—movies, travel plans, random memories.
    It felt refreshing to be around someone who doesn’t expect anything from me.
    Connection doesn’t always need depth; sometimes presence is enough.""",

    """Spent the evening chatting online with my old school friend.
    We laughed about how serious we used to be.
    It’s funny how friendship can survive years of silence and still feel natural once it restarts.""",

    # WORK
    """Work was nonstop today—emails, meetings, small fires everywhere.
    I kept switching tasks but never felt like I actually finished anything.
    I really need to rethink how I organize my day.
    The exhaustion hits harder when there’s no visible progress.""",

    """Started a new project at work that feels genuinely exciting.
    The planning session went smoothly, and everyone seemed aligned.
    For once, I’m looking forward to building something rather than just maintaining it."""
]

In [2]:
from bertopic import BERTopic
topic_model = BERTopic(language="english", min_topic_size=2)
topics, probs = topic_model.fit_transform(journals)
topic_model.get_topic_info()

C:\Anass_de_programmeur\School\Reflect_Audio_Journaling\Research\topic_segmentation_multiple_journals\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Topic,Count,Name,Representation,Representative_Docs
0,-1,2,-1_doesnt_from_and_in,"[doesnt, from, and, in, made, with, felt, the,...",[Met up with Lucas for coffee after weeks of n...
1,0,6,0_the_me_but_to,"[the, me, but, to, my, how, and, was, that, like]","[I cooked a big pasta meal tonight, just me an..."
2,1,2,1_once_we_it_how,"[once, we, it, how, to, and, exciting, everyon...",[Started a new project at work that feels genu...


In [5]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-MiniLM-L6-v2")

labels = [
    "food and cooking experiences",
    "exercise, training, physical activity",
    "sleep, rest, and recovery",
    "friends and social connections",
    "work, projects, and career",
    "relaxation or personal downtime"
]

label_embeds = model.encode(labels, convert_to_tensor=True)

semantic_labels = {}
for topic_id, words in semantic_labels.items():
    topic_embed = model.encode(" ".join(words), convert_to_tensor=True)
    sim = util.cos_sim(topic_embed, label_embeds)
    best_label = labels[sim.argmax().item()]
    semantic_labels[topic_id] = best_label

In [6]:
# 1. after you have topics and journals
import torch
from torch import mean

topic_embeds = {}
for topic_id in set(topics):
    entries = [journals[i] for i, t in enumerate(topics) if t == topic_id]
    if not entries:
        continue
    entry_embeds = model.encode(entries, convert_to_tensor=True)
    topic_embeds[topic_id] = torch.mean(entry_embeds, dim=0)

# 2. now assign semantic labels
semantic_labels = {}
for topic_id, emb in topic_embeds.items():
    sim = util.cos_sim(emb, label_embeds)[0]
    sorted_indices = sim.argsort(descending=True)
    top1, top2 = sorted_indices[:2]
    if sim[top1] - sim[top2] < 0.05:
        best_label = f"{labels[top1]} / {labels[top2]}"
    else:
        best_label = labels[top1]
    semantic_labels[topic_id] = best_label

# 3. print results
for topic_id, label in semantic_labels.items():
    entries = [journals[i] for i, t in enumerate(topics) if t == topic_id]
    print(f"Topic: {label}")
    for j in entries:
        print("  -", j)


Topic: sleep, rest, and recovery
  - I cooked a big pasta meal tonight, just me and the quiet kitchen.
    The smell of garlic and olive oil filled the room.
    Eating alone felt comfortable rather than sad, which surprised me.
    Maybe this is what self-care actually looks like.
  - I forced myself to go for a morning run before work.
    The first few minutes were rough, but once my breathing found a rhythm, everything felt lighter.
    The sunrise over the river almost made me forget how tired I was.
    It reminded me that most progress starts with discomfort.
  - Today’s workout was heavy on strength training.
    My arms are sore, but it’s the kind of ache that tells me I did something right.
    I tracked my lifts and saw steady improvement compared to last week.
    Discipline feels like a quiet kind of reward.
  - I finally let myself sleep in without feeling guilty.
    Woke up late and stretched slowly under the blanket.
    The extra hours made me realize how exhausted I’

In [7]:
for topic_id, label in semantic_labels.items():
    entries = [journals[i] for i, t in enumerate(topics) if t == topic_id]
    print(f"Topic: {label}")
    for j in entries:
        print("  -", j)

Topic: sleep, rest, and recovery
  - I cooked a big pasta meal tonight, just me and the quiet kitchen.
    The smell of garlic and olive oil filled the room.
    Eating alone felt comfortable rather than sad, which surprised me.
    Maybe this is what self-care actually looks like.
  - I forced myself to go for a morning run before work.
    The first few minutes were rough, but once my breathing found a rhythm, everything felt lighter.
    The sunrise over the river almost made me forget how tired I was.
    It reminded me that most progress starts with discomfort.
  - Today’s workout was heavy on strength training.
    My arms are sore, but it’s the kind of ache that tells me I did something right.
    I tracked my lifts and saw steady improvement compared to last week.
    Discipline feels like a quiet kind of reward.
  - I finally let myself sleep in without feeling guilty.
    Woke up late and stretched slowly under the blanket.
    The extra hours made me realize how exhausted I’